In [1]:
import pandas as pd
import numpy as np
import gc
from time import time
import math
import random

import datetime
import pkg_resources
#import seaborn as sns
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Subset, DataLoader
from torch.optim import Optimizer

#from nltk.stem import PorterStemmer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
#%load_ext autoreload
#%autoreload 2
#%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
#from apex import amp
import shutil


In [2]:
device=torch.device('cuda')

In [3]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class AdamW(Optimizer):
    """Implements AdamW algorithm.

    It has been proposed in `Fixing Weight Decay Regularization in Adam`_.

    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)

    .. Fixing Weight Decay Regularization in Adam:
    https://arxiv.org/abs/1711.05101
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # according to the paper, this penalty should come after the bias correction
                # if group['weight_decay'] != 0:
                #     grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                if group['weight_decay'] != 0:
                    p.data.add_(-group['weight_decay'], p.data)

        return loss


In [4]:
MAX_SEQUENCE_LENGTH = 295 
SEED = 42
EPOCHS = 20
Data_dir="../job_nlp/"
WORK_DIR = "../job_nlp/working/"
#num_to_load=100000                         #Train size to match time limit
#valid_size= 50000                          #Validation Size
TARGET = 'smishing'

In [5]:
# https://www.kaggle.com/matsuik/ppbert
package_dir_a = "../job_nlp/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

In [6]:
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py:5879: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/yilgukseo/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))


In [7]:
# Translate model from tensorflow to pytorch
BERT_MODEL_PATH = '../job_nlp/bert-pretrained-models/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

erNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_9/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_9/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_9/output/dense/kernel with shape [3072, 768]
Loading TF weight bert/pooler/dense/bias with shape [768]
Loading TF weight bert/pooler/dense/kernel with shape [768, 768]
Loading TF weight cls/predictions/output_bias with shape [30522]
Loading TF weight cls/predictions/transform/LayerNorm/beta with shape [768]
Loading TF weight cls/predictions/transform/LayerNorm/gamma with shape [768]
Loading TF weight cls/predictions/transform/dense/bias with shape [768]
Loading TF weight cls/predictions/transform/dense/kernel with shape [768, 768]
Loading TF weight cls/seq_relationship/output_bias with shape [2]
Loading TF weight cls/seq_relationship/output_weights with shape [2, 768]
Initialize PyTorch weight ['bert', 'embeddings', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert',

'../job_nlp/working/bert_config.json'

In [8]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../job_nlp/bert-pretrained-models/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/'+'bert_config.json')

In [9]:
bert_config

{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [10]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [11]:
BERT_MODEL_PATH = '../job_nlp/bert-pretrained-models/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/'

In [12]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [13]:
%%time
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))
test_df = pd.read_csv(os.path.join(Data_dir,"test.csv"))

CPU times: user 31.9 ms, sys: 0 ns, total: 31.9 ms
Wall time: 31.7 ms


In [14]:
train_df.label.value_counts(normalize=True)

3    0.354552
5    0.248457
4    0.184414
2    0.113812
1    0.098765
Name: label, dtype: float64

In [15]:
# replace NaN
print(train_df.iloc[2400,:])
train_df.iloc[2400,1] = "@@@"

title      [$$$] 재귀함수를 처리하면서 setTimeout으로 딜레이를 주면 읽는 순서가 ...
content                                                  NaN
label                                                      4
Name: 2400, dtype: object


In [16]:
%%time
train_df['text'] = train_df[['title', 'content']].apply(lambda x: ' '.join(x), axis = 1)
test_df['text'] = test_df[['title', 'content']].apply(lambda x: ' '.join(x), axis = 1) 

CPU times: user 39.5 ms, sys: 0 ns, total: 39.5 ms
Wall time: 39.3 ms


In [17]:
train_df.head()

,title,content,label,text
0,wait()과 sleep()의 차이점은 뭔가요,###발생하는 문제 및 실행환경\nwait()과 sleep()의 차이점은 뭔가요,3,wait()과 sleep()의 차이점은 뭔가요 ###발생하는 문제 및 실행환경\nw...
1,$$$ 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠㅠ,"1.헤더에, 사용할 멤버변수가 담긴 헤더 파일이 Include 되어있습니다.\n예를...",2,"$$$ 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠㅠ 1.헤더에, 사용할 멤버변..."
2,ORM: Sequelize: 다대다 관계 쿼리,"안녕하세요.\n\n어떻게 다대다 관계 쿼리를 해야하나요? 예를들어, `product...",4,ORM: Sequelize: 다대다 관계 쿼리 안녕하세요.\n\n어떻게 다대다 관계...
3,$$$에서 숫자가 NaN인지 검사하려면 어떻게해야하죠?,```\nparseFloat('geoff') == NaN;\n\nparseFloat...,4,$$$에서 숫자가 NaN인지 검사하려면 어떻게해야하죠? ```\nparseFloat...
4,"$$$ 2.7에서 중국어, 특수문자 인코딩 하는 방법이 뭔가요?",```\n>>>dict['name']\n胡安·马塔\n>>>json.dumps(dic...,5,"$$$ 2.7에서 중국어, 특수문자 인코딩 하는 방법이 뭔가요? ```\n>>>di..."


In [18]:
len(train_df.iloc[0,0])

25

In [19]:
len(train_df.iloc[0,1])

43

In [20]:
len(train_df.iloc[0,3])

69

In [21]:
train_df['t_length'] = train_df['title'].apply(lambda x: len(x))
train_df['c_length'] = train_df['content'].apply(lambda x: len(x))
train_df['text_length'] = train_df['text'].apply(lambda x: len(x))

test_df['t_length'] = test_df['title'].apply(lambda x: len(x))
test_df['c_length'] = test_df['content'].apply(lambda x: len(x))
test_df['text_length'] = test_df['text'].apply(lambda x: len(x))

In [22]:
train_df.describe()

,label,t_length,c_length,text_length
count,2592.000000,2592.000000,2592.000000,2592.000000
mean,3.369985,29.945988,891.706404,922.652392
std,1.245321,15.235909,1684.607332,1685.667323
min,1.000000,3.000000,3.000000,8.000000
25%,3.000000,19.000000,180.000000,208.000000
50%,3.000000,27.000000,385.000000,418.500000
75%,4.000000,37.000000,896.000000,920.750000
max,5.000000,119.000000,28756.000000,28780.000000


In [23]:
test_df.describe()

,t_length,c_length,text_length
count,500.000000,500.000000,500.000000
mean,29.310000,858.270000,888.580000
std,14.786046,1957.436752,1956.747171
min,5.000000,20.000000,53.000000
25%,19.750000,192.250000,227.250000
50%,26.500000,371.000000,399.000000
75%,36.000000,841.250000,870.500000
max,134.000000,35210.000000,35226.000000


In [24]:
MAX_SEQUENCE_LENGTH = 400

### Tokenizing

In [25]:
#%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
#train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))#.sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df))
# Make sure all comment_text values are strings
train_df['text'] = train_df['text'].astype(str) 
x_train = convert_lines(train_df["text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
print("X_train : {}".format(len(x_train)))

#test_df = pd.read_csv(os.path.join(Data_dir,"public_test.csv"))#.sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(test_df))
test_df['text'] = test_df['text'].astype(str) 
x_test = convert_lines(test_df["text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
print("X_test : {}".format(len(x_test)))

train_df=train_df.fillna(0)

loaded 2592 records
/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



1065
X_train : 2592
loaded 500 records



208
X_test : 500


In [26]:
# above not working in linux ?? these x_train & x_test are obtained from windows
#x_train = np.loadtxt('../job_nlp/x_train.csv', delimiter=',')
#x_test = np.loadtxt('../job_nlp/x_test.csv', delimiter=',')

In [27]:
seed_everything(SEED)
output_model_file = "bert_pytorch.bin"

lr=2e-5
batch_size = 8
accumulation_steps=2
n_labels = 2

criterion = nn.CrossEntropyLoss()

TARGET = 'label'

In [28]:
train_df[TARGET] = train_df[TARGET]-1

In [29]:
#x_train = train_df['text']
y_train = torch.tensor(train_df[TARGET])#.long()
y_train

tensor([2, 1, 3,  ..., 0, 4, 1])

In [30]:
y_train[:5]

tensor([2, 1, 3, 3, 4])

In [31]:
def to_numpy(x):
    return x.cpu().detach().numpy()

In [32]:
test_dataset = TensorDataset(torch.tensor(x_test, dtype = torch.long)) #TensorDataset(X_valid, valid_length, torch.tensor(Y_valid))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [33]:
model = BertForSequenceClassification.from_pretrained("../job_nlp/working",cache_dir=None, num_labels=5)

In [34]:
%%time
best_epoch_list = []
best_val_acc_list = []
start_time = time()

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(x_train, y_train))
for fold in [0, 1, 2, 3, 4]:

    print("{}/5 fold training starts!".format(fold+1))
    
    fold_num = str(fold + 1)

    trn_index, val_index = splits[fold]

    X_train, X_valid = x_train[trn_index], x_train[val_index]
    #train_length, valid_length = lengths[trn_index], lengths[val_index]
    Y_train, Y_valid = y_train[trn_index], y_train[val_index]

    train_dataset = TensorDataset(torch.tensor(X_train, dtype = torch.long), torch.tensor(Y_train, dtype=torch.long)) #TensorDataset(X_train, train_length, torch.tensor(Y_train))
    valid_dataset = TensorDataset(torch.tensor(X_valid, dtype = torch.long), torch.tensor(Y_valid, dtype=torch.long)) #TensorDataset(X_valid, valid_length, torch.tensor(Y_valid))
    
    model = BertForSequenceClassification.from_pretrained("../job_nlp/working",cache_dir=None, num_labels=5)
    model.zero_grad()
    model = model.to(device)
    #optimizer = BertAdam(optimizer_grouped_parameters,
    #                 lr=lr,
    #                 warmup=0.05,
    #                 t_total=num_train_optimization_steps)
    #scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    #train = train_dataset

    num_train_optimization_steps = int(EPOCHS*len(train_dataset)/batch_size/accumulation_steps)

    #optimizer = BertAdam(optimizer_grouped_parameters,
    #                     lr=lr,
    #                     warmup=0.05,
    #                     t_total=np.ceil(num_train_optimization_steps))

    optimizer = AdamW(model.parameters(), lr, weight_decay=0.000025)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        
    best_valid_score = 0
    best_val_acc = 0
    
    #tq = tqdm_notebook(range(EPOCHS))
    #model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    for epoch in range(1, EPOCHS + 1):
        
        #start_time = time.time()
        train_loss = 0
        train_total_correct = 0
        model.train()
        optimizer.zero_grad()
        #tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
        
        for i, (x_batch, y_batch) in enumerate(train_loader):
            preds  = model(x_batch.to(device), attention_mask = (x_batch>0).to(device), labels=None)
            loss = criterion(preds, y_batch.to(device))
            loss.backward()
            
            if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
                optimizer.step()                            # Now we can do an optimizer step
                optimizer.zero_grad()
            else:
                optimizer.step()
                optimizer.zero_grad()

            train_loss += loss.item()/len(train_loader)
            
        # Validation Starts
        model.eval()
        val_loss = 0
        valid_total_correct = 0
        
        #valid_preds = np.zeros(len(valid_dataset),5)
        #valid_targets = np.zeros(len(valid_dataset),5)
        
        with torch.no_grad():
            for i, (x_batch, y_batch) in enumerate(valid_loader):
                #valid_targets[i*batch_size: (i+1)*batch_size] = y_batch.numpy().copy()
                
                preds = model(x_batch.to(device), attention_mask = (x_batch>0).to(device), labels=None)
                
                loss = criterion(preds, y_batch.to(device))
                
                output_prob = F.softmax(preds, dim=1)

                predict_vector = np.argmax(to_numpy(output_prob), axis=1)
                label_vector = to_numpy(y_batch)
                #valid_preds[i*batch_size: (i+1)*batch_size] = np.argmax(preds_prob.detach().cpu().squeeze().numpy())
                bool_vector = predict_vector == label_vector
                
                val_loss += loss.item()/len(valid_loader)
                valid_total_correct += bool_vector.sum()
        
        #val_score = roc_auc_score(valid_targets, valid_preds)

        elapsed = time() - start_time
        val_acc = valid_total_correct / len(valid_loader.dataset)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch
            print("val_acc has improved !! ")
            best_epoch_list.append(best_epoch)
            best_val_acc_list.append(best_val_acc)
            
            torch.save(model.state_dict(), '../job_nlp/Bert_20e_maxseq400_fold_{}.pt'.format(fold))
            #print("================ ༼ つ ◕_◕ ༽つ BEST epoch : {}, Accuracy : {} ".format(epoch, best_val_acc))
            
        #lr = [_['lr'] for _ in optimizer.param_g] # or optimizer
        print("================ ༼ つ ◕_◕ ༽つ Epoch {} - train_loss: {:.5f}  val_loss: {:.5f}  val_acc: {:.5f}  elapsed: {:.0f}m {:.0f}s".format(epoch, train_loss, val_loss, best_val_acc, elapsed // 60, elapsed % 60))
    print("============== ༼ つ ◕_◕ ༽つ BEST epoch : {}, Accuracy : {} ====================================".format(epoch, best_val_acc))
    #best_epoch_list.append(best_epoch)
    #best_val_acc_list.append(best_val_acc)
    
    #---- Inference ----
    #batch_size = 8

    print("========================== ༼ つ ◕_◕ ༽つ Model Load {}_th FOLD =================================".format(fold))
    model.load_state_dict(torch.load('Bert_20e_maxseq400_fold_{}.pt'.format(fold)))
    model.eval()
    predictions = np.zeros((len(test_loader.dataset),5))
    with torch.no_grad():
        for i, (x_batch, ) in enumerate(test_loader):
            preds = model(x_batch.to(device), attention_mask = (x_batch>0).to(device), labels=None)
            
            predictions[i*batch_size: (i+1)*batch_size] = to_numpy(preds)
    print("predict values check : ",predictions[0])
    np.savetxt("../job_nlp/bert_raw_submission/bert_20e_maxseq400_fold_{}.csv".format(fold), predictions, delimiter=",")   

1/5 fold training starts!
/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ Epoch 1 - train_loss: 1.30095  val_loss: 1.10379  val_acc: 0.56069  elapsed: 1m 29s
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ Epoch 2 - train_loss: 1.01166  val_loss: 0.84731  val_acc: 0.69557  elapsed: 2m 55s
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ Epoch 3 - train_loss: 0.70330  val_loss: 0.69070  val_acc: 0.75915  elapsed: 4m 24s
================ ༼ つ ◕_◕ ༽つ Epoch 4 - train_loss: 0.52543  val_loss: 0.69923  val_acc: 0.75915  elapsed: 5m 54s
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ Epoch 5 - train_loss: 0.39848  val_loss: 0.64605  val_acc: 0.76686  elapsed: 7m 20s
================ ༼ つ ◕_◕ 